# Keystone Project - Algorithmic Trading

__Assignment / Goal:__

Create a Trading Algorithm for the following (simple) Contrarian Trading Strategy: 

1. At the end of each trading day, you __buy__ the __three worst performing stocks__ among the __30 DJIA__ stocks and hold them for the next trading day.

2. At the end of each trading day, you __short sell__ the __three best performing stocks__ among the __30 DJIA__ stocks and short them for the next trading day.

3. For simplicity reasons, you buy/sell __one share__ of the respective stocks.

4. Underlying idea: The __Market overreacted__ and most recent (extreme) price __trends will revert__ in the near future -> __Contrarian Trading__ (!= Momentum Trading)

5. Develop the code that
- __imports__ required data 
- __measures__ the most recent performance (price change in %)
- __identifies__ the best/worst performing stocks
- __trades__ the stocks with the IBKR API 

### -------------SOLUTION---------------

__Please run the following code only with your Paper Trading Account!!!__

__Check the Regular Trading Hours!!!__ (trade right before 16:30 US Eastern time)

In [ ]:
import pandas as pd
import yfinance as yf
from ib_insync import *
util.startLoop()

## Get the Symbols

In [ ]:
df = pd.read_csv("DJI_Const.csv", header = [0, 1], index_col = 0, parse_dates = [0])
df

In [ ]:
symbols = df.Close.columns.to_list()
symbols

In [ ]:
symbols.remove("^DJI")

In [ ]:
len(symbols)

## Get most recent Prices and Performance

__Alternative 1__

In [ ]:
# yf.Ticker(ticker = "AAPL").get_info()["regularMarketPrice"] # OLD

In [ ]:
cprice = yf.Ticker(ticker = "AAPL").get_info()["currentPrice"] # NEW (before "regularMarketPrice")
cprice

In [ ]:
last_close = yf.Ticker("AAPL").get_info()["regularMarketPreviousClose"]
last_close

__Alternative 2__ (faster, more reliable)

In [ ]:
yf.Ticker(ticker = "AAPL").get_fast_info() 

In [ ]:
cprice =  yf.Ticker("AAPL").get_fast_info()["last_price"]
cprice

In [ ]:
last_close = yf.Ticker("AAPL").get_fast_info()["regularMarketPreviousClose"] 
last_close

In [ ]:
perf = cprice / last_close - 1
perf

In [ ]:
perf = pd.Series(dtype = float)
perf

In [ ]:
symbols

In [ ]:
count = 1
for symbol in symbols:
    try:
        fast_info = yf.Ticker(ticker = symbol).get_fast_info() 
        prc_chg = fast_info["last_price"] / fast_info["regularMarketPreviousClose"] - 1 
        perf.loc[symbol] = prc_chg
        print(count, end = '\r')
        count += 1
    except Exception as e:
        print("{} not found".format(symbol))
print("Download complete.")

In [ ]:
perf

In [ ]:
perf.sort_values(inplace = True)
perf

In [ ]:
perf.index.name = "symbol"
perf 

## Determine Target Positions

In [ ]:
buy_stocks = 3 # buy the 3 worst performing stocks

In [ ]:
sell_stocks = 3 # short sell the 3 best performing stocks

In [ ]:
shares = 1 # one share per stock

In [ ]:
perf.iloc[:buy_stocks] = shares
perf.iloc[-sell_stocks:] = -shares
perf

In [ ]:
target = pd.concat([perf.iloc[:buy_stocks], perf.iloc[-sell_stocks:]]).to_frame().reset_index()
target.columns = ["symbol", "position"]
target

## Identify Current Positions

In [ ]:
ib = IB()

In [ ]:
ib.connect()

In [ ]:
pos = ib.positions()
pos

In [ ]:
df = util.df(pos)
df

In [ ]:
#df["symbol"] = df.contract.apply(lambda x: x.symbol)
#df["conID"] = df.contract.apply(lambda x: x.conId)

In [ ]:
if df is not None:
    df["symbol"] = df.contract.apply(lambda x: x.symbol)
    df["conID"] = df.contract.apply(lambda x: x.conId)
else: 
    df = pd.DataFrame(columns = ["symbol", "position"])

In [ ]:
df

## Determine Required Trades (from actual to target positions)

In [ ]:
target

In [ ]:
df

In [ ]:
trades = pd.merge(target, df[["symbol", "position"]], "outer", on = "symbol", suffixes = ["_t", "_a"])
trades

In [ ]:
trades.fillna(0, inplace = True)
trades

In [ ]:
trades["trades"] = trades.position_t - trades.position_a 

In [ ]:
trades

In [ ]:
trades = trades[trades.trades !=0].set_index("symbol").copy()
trades

## Execute Trades

In [ ]:
for symbol in trades.index:
    to_trade = trades.loc[symbol, "trades"]
    if to_trade > 0: 
        side = "BUY"
    elif to_trade < 0:
        side = "SELL"
    contract = Stock(symbol, "SMART", "USD")
    cds = ib.reqContractDetails(contract)
    if len(cds) == 0:
        print("No Contract for {} found.".format(symbol))
    elif len(cds) == 1:
        contract = cds[0].contract
        order = MarketOrder(side, abs(to_trade))
        trade = ib.placeOrder(contract, order)
        while not trade.isDone():
            ib.waitOnUpdate()
        if trade.orderStatus.status == "Filled":
            print("{} {} @ {}".format(side, symbol, trade.orderStatus.avgFillPrice))
        else:
            print("{} {} failed.".format(side, symbol))
    else:
        contract = cds[0].contract
        print("Multiple Contracts for {} found.".format(symbol))
        order = MarketOrder(side, abs(to_trade))
        trade = ib.placeOrder(contract, order)
        while not trade.isDone():
            ib.waitOnUpdate()
        if trade.orderStatus.status == "Filled":
            print("{} {} @ {}".format(side, symbol, trade.orderStatus.avgFillPrice))
        else:
            print("{} {} failed.".format(side, symbol))  
pos = ib.positions()
df = util.df(pos)
df["symbol"] = df.contract.apply(lambda x: x.symbol)
df["conID"] = df.contract.apply(lambda x: x.conId)
df

In [ ]:
target

In [ ]:
ib.disconnect()

## Running a Python Script

__Important things to consider__:
- You should run the __full code__ right before the end of the trading day (__16:30 US/Eastern__) 
- better/faster execution with a __Python Script__
- __Daily Automation/Schedule__ easier with Script (not part of this Section)
- Live Trading only after __extensive Strategy Backtesting__ (see Part 3)!
    
    